In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('/Users/fahad/Desktop/processed_data/combined_Top.parquet')

In [17]:
dropped_cols = ['id', 'conversationId','source','sourceUrl','sourceLabel', 'cashtags', 'vibe', 'inReplyToTweetId', 'viewCount', 'retweetedTweet', 'links', 'lang', 'media', 'quotedTweet', 'inReplyToUser', 'mentionedUsers', 'card' ]

df.drop(columns=['lang', 'media', 'quotedTweet', 'inReplyToUser', 'mentionedUsers', 'card' ], inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2658 entries, 0 to 2657
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   url              2658 non-null   object             
 1   date             2658 non-null   datetime64[ns, UTC]
 2   rawContent       2658 non-null   object             
 3   renderedContent  2658 non-null   object             
 4   user             2658 non-null   object             
 5   replyCount       2658 non-null   int64              
 6   retweetCount     2658 non-null   int64              
 7   likeCount        2658 non-null   int64              
 8   quoteCount       2658 non-null   int64              
 9   coordinates      134 non-null    object             
 10  place            134 non-null    object             
 11  hashtags         2657 non-null   object             
 12  filename         2658 non-null   object             
 13  filepath         2

In [40]:
user = list(df['user'])

In [42]:
user_df = pd.json_normalize(user)

In [36]:
user_df_dropped_cols = ['descriptionLinks', 'displayname', 'favouritesCount', 'id', 'label', 'listedCount', 'profileBannerUrl', 'profileImageUrl', 'protected', 'username', 'statusesCount', 'link.indices', 'link.tcourl', 'link.text', 'link.url', 'link', 'label.badgeUrl', 'label.description', 'label.longDescription', 'label.url']
user_df.drop([], inplace=True, axis=1)

In [38]:
df = pd.concat([df, user_df], axis=1)

In [44]:
df.drop('user', inplace=True, axis=1)

In [48]:
coordinates = list(df['coordinates'])

In [51]:
coordinates_df = pd.json_normalize(coordinates)